# Plot Hubble diagram

# User interface

In [38]:
# Peculiar velocity. This is used only in the part where I compute
# the intrinsic dispersion. 150 km/s is the value I used to GP fit each SN
sigma_vpec = 150     # km/s. Usual options (150, 200, 300, 400)

#--------------------------------------------

# Samples to plot: 
LowzSample=False
raisin_1=False
raisin_2=False
raisin_1_2 = True

# LOW-Z sub-sample:
# Sample = 'CfA'
# Sample = 'CSP'
# Sample = 'Others'
Sample = 'AllSamples'

BandType = 'Optical'
# BandType = 'OpticalNIR'

#----------------------

PrintNameOutliers = True
PrintRMS = False

################################
# (Fixed) 

# Intervals for plotting the Hubble diagram

# x_interval = (0, 0.065)
# x_interval = (0, 0.7)

# y_interval = (-1.2, 1.2)
y_interval = (-1, 1)

#----------------------

# Hubble constant (km/(s Mpc)):
#    - 70 : SNANA-SALT2 default value
#    - 72 : SNoopy  default value
#    - 73.24: Adam Riess Cepheid values. This is what I use in the low-z paper.

# SNANA-SALT2 default values:
HoFix = 70.0 
OmMFix = 0.3 # Omega_Matter
OmLFix = 0.7 # Omega_Lambda
wFix = -1.0 # Dark energy EoS

# SNooPy default values
# HoFix = 72  
# OmMFix = 0.28 # Omega_Matter
# OmLFix = 0.72 # Omega_Lambda
# wFix = -1 # Dark energy EoS

#----------------------

# Quality Cutoffs for the low-z sample
zcmbMin = 0.0;  zcmbMax = 0.06
dm15Min = 0.80;  dm15Max = 1.60
EBVhostMax = 0.4
EBV_mwMax  = 1.0

################################

#--- Fixed values ---
cc = 299792.458  # Speed of light (km/s)

#--- Uncertainty in z_CMB:---

# Dan Scolnic gave me the value of err_cz_CMB = 150 km/s about the collection 
# of z_CMB values he provided me.
err_cz_CMB = 150 # km/s. 

err_zCMB_fix = err_cz_CMB/cc

In [39]:
# Defining the directories

print '# Directories of the distance-modulus files for the low-z sample:'

#----------- RAISIN ------------

if raisin_1_2:
    file_raisin_1_2 = 'DistanceMu_Good_AfterCutoffs_Main_.txt'
    dir_raisin_1_2 = '/Users/arturo/Dropbox/Research/\
Articulos/14_RAISINs/Data/raisin12/hubblediagram/salt2_optical_v2/'
    print "# Directory RAISIN 1-2:"
    print "%s"%dir_raisin_1_2

if raisin_1:
    Dir_raisin_1 = 'Data/RAISIN_1/Data/2017/Snoopy/'+BandType+'/Fit/'
    
if raisin_2:
    Dir_raisin_2 = 'Data/RAISIN_2/Data/2017_03_30/Snoopy_zp27.5/'+BandType+'/Fit/'

#----------- LOW-Z ------------

if LowzSample:
    DirLowzData = '/Users/arturo/Dropbox/Research/SoftwareResearch/Snoopy/AndyLCComp/\
    all/Snoopy_FromMag/2_4RAISINs_old/' 
    DirMuData_lowz = DirLowzData+Sample+'/'+BandType+'/'

#----------- SAVE OUTPUT ------------

DirSaveOutput= dir_raisin_1_2+'plots_HDs_/'

#- Force the creation of the directory to save the outputs.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions
if not os.path.exists(DirSaveOutput): os.makedirs(DirSaveOutput)
    
print '\n Dir to save the outputs:'
print DirSaveOutput


# Directories of the distance-modulus files for the low-z sample:
# Directory RAISIN 1-2:
/Users/arturo/Dropbox/Research/Articulos/14_RAISINs/Data/raisin12/hubblediagram/salt2_optical_v2/

 Dir to save the outputs:
/Users/arturo/Dropbox/Research/Articulos/14_RAISINs/Data/raisin12/hubblediagram/salt2_optical_v2/plots_HDs_/


------

# Automatic

In [40]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import quad as intquad
from matplotlib import gridspec

# import os
# import glob # To read the name of the files in a given directory

5+4

9

#### Read the data

In [41]:
# Read the distance modulus file

if raisin_1_2:

    mu_data_np = np.genfromtxt(dir_raisin_1_2+file_raisin_1_2, 
                    dtype=['S32', float,float,float,float,float,float,
                                  float,float,float,float,float])

""" 
Mu_lowz = np.genfromtxt(DirMuData_lowz+'DistanceModulus_.txt', 
                dtype=['S32', float,float,float,float,float,float,float,float,
                              float,float,float,float,float,float,float,float])

Mu_raisin1 = np.genfromtxt(Dir_raisin_1+'DistanceModulus_.txt', 
                dtype=['S32', float,float,float,float,float,float,float,float,
                              float,float,float,float,float,float,float,float])

Mu_raisin2 = np.genfromtxt(Dir_raisin_2+'DistanceModulus_.txt', 
                dtype=['S32', float,float,float,float,float,float,float,float,
                              float,float,float,float,float,float,float,float])



Mu_raisin1Snana = np.genfromtxt(Dir_raisin1_snana+'SALT2mu.fitres', skip_header=2,
                dtype=['S4', 'S13', 
                      float, float, float, float, float, float, float, float, float, float,
                      float, float, float, float, float, float, float, float, float, float,
                      float, float, float, float, float, float, float, float, float])  """
0

0

In [42]:
mu_data_np[0]

('snPSc440005', 0.306, 0.001, 40.98658, 0.12904, -0.018525, 0.61, 2., 21.9781, 0.0484, 41.00510104, 0.00793502)

## The theory

In [43]:
# Inverse of the dimensionless Hubble parameter
def InvEHubblePar(z, OmM, wde):
    "Dimensionless Hubble parameter"
    InvEHubbleParInt = 1.0/(np.sqrt(OmM*(1.0+z)**3.0 + (1.0-OmM)*(1.+z)**(3.*(1.+wde))))
    return InvEHubbleParInt

# ---- The luminosity distance ----
def LumDistanceVec(z, OmM, wde, Ho):
    "Luminosity distance"
    LumDistanceVecInt = 0.
    LumDistanceVecInt = cc*(1.+z)*intquad(InvEHubblePar, 0., z, args=(OmM, wde))[0]/Ho 
    return LumDistanceVecInt

# ---- Distance modulus scalar ----
def DistanceMu(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt = 5.0*np.log10(LumDistanceVec(z, OmM, wde, Ho)) + 25.0
    return DistanceMuInt

# ---- Distance modulus Vector ----
def DistanceMuVector(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt= []
    for i in range(len(z)):
        DistanceMuInt += [5.0*np.log10(LumDistanceVec(z[i], OmM, wde, Ho)) + 25.0] 
    return DistanceMuInt

#--------------------------------------------------

ztest1 = 0.01

print 'Checking that the functions work well:', DistanceMu(ztest1, OmMFix, wFix, HoFix)
# Checking that the functions work well: 33.1144718537

Checking that the functions work well: 33.17531838091242


In [44]:
# sigma^2_mu from the peculiar velocity uncertainty

def sigma2_mu_vpec(zcmb, err_zcmb, sigma_vpec):
    sigma2_mu_vpecInt = ((5/(zcmb*np.log(10.0)))*np.sqrt((sigma_vpec/cc)**2 + 
                                                         err_zcmb**2))**2
    return sigma2_mu_vpecInt

# Test
sigma2_mu_vpec(0.0109942726, 0.0010420420, 150)
# 0.052125037354329877

0.05212503735432988

-------

# Plot Hubble diagram and residual

In [45]:
# Define the input data as numpy arrays

zcmb_np = mu_data_np['f1']
mu_np = mu_data_np['f3']
err_mu_np = mu_data_np['f4']
res_mu_np = mu_data_np['f5']
sampleflag_np = mu_data_np['f7']

### Main: plot

In [30]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
%autoreload 1

In [49]:
%aimport plotHubbleDiagram

In [85]:
# Run my function to create the Hubble diagram and residual plots

plotHubbleDiagram.plotHD(zcmb_np, mu_np, err_mu_np, res_mu_np, sampleflag_np, 
                         nameoutput= 'MyFile', saveinEPS=False,
                         dirsave='/Users/arturo/Dropbox/Research/Articulos/14_RAISINs/')

In [88]:
plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();

In [87]:
?plotHubbleDiagram.plotHD

Signature: plotHubbleDiagram.plotHD(zcmb, mu, err_mu, res_mu, sampleflag, nameoutput='MyFile.png', dirsave='', title='Hubble diagram', saveinEPS=False, **kwargs)
Docstring:
:title: title of the plot to write on top of the figure.
:saveinEPS: Save the plot in EPS format too.
:param f: function to plot
:type f: callable
:param x: values for x
:type x: list or ndarray

Plots the distance moduli and residuals data in a Hubble diagram.
File:      ~/Dropbox/Research/Articulos/14_RAISINs/Codes/github/RAISIN/plotHubbleDiagram.py
Type:      function


In [76]:
help(np.genfromtxt)

Help on function genfromtxt in module numpy.lib.npyio:

genfromtxt(fname, dtype=<type 'float'>, comments='#', delimiter=None, skip_header=0, skip_footer=0, converters=None, missing_values=None, filling_values=None, usecols=None, names=None, excludelist=None, deletechars=None, replace_space='_', autostrip=False, case_sensitive=True, defaultfmt='f%i', unpack=None, usemask=False, loose=True, invalid_raise=True, max_rows=None, encoding='bytes')
    Load data from a text file, with missing values handled as specified.
    
    Each line past the first `skip_header` lines is split at the `delimiter`
    character, and characters following the `comments` character are discarded.
    
    Parameters
    ----------
    fname : file, str, pathlib.Path, list of str, generator
        File, filename, list, or generator to read.  If the filename
        extension is `.gz` or `.bz2`, the file is first decompressed. Note
        that generators must return byte strings in Python 3k.  The strings
    

In [77]:
help(plt.plot)

Help on function plot in module matplotlib.pyplot:

plot(*args, **kwargs)
    Plot lines and/or markers to the
    :class:`~matplotlib.axes.Axes`.  *args* is a variable length
    argument, allowing for multiple *x*, *y* pairs with an
    optional format string.  For example, each of the following is
    legal::
    
        plot(x, y)        # plot x and y using default line style and color
        plot(x, y, 'bo')  # plot x and y using blue circle markers
        plot(y)           # plot y using x as index array 0..N-1
        plot(y, 'r+')     # ditto, but with red plusses
    
    If *x* and/or *y* is 2-dimensional, then the corresponding columns
    will be plotted.
    
    If used with labeled data, make sure that the color spec is not
    included as an element in data, as otherwise the last case
    ``plot("v","r", data={"v":..., "r":...)``
    can be interpreted as the first case which would do ``plot(v, r)``
    using the default line style and color.
    
    If not used wi